# TwitchLeagueBERT

### VM setup

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd "/content/gdrive/MyDrive/Master Thesis/Language Model Training"

/content/gdrive/MyDrive/Master Thesis/Language Model Training


In [3]:
!pip install transformers==4.17

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.8 MB 4.0 MB/s 
     |████████████████████████████████| 596 kB 37.1 MB/s 
     |████████████████████████████████| 101 kB 10.2 MB/s 
     |████████████████████████████████| 6.6 MB 41.0 MB/s 
     |████████████████████████████████| 880 kB 57.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=c65c10d7442c3556fac9d04d357f4de99e499ecaf6b998b593253b23d0df34d7
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.1 MB/s 
     |████████████████████████████████| 141 kB 46.1 MB/s 
     |████████████████████████████████| 115 kB 59.2 MB/s 
     |████████████████████████████████| 212 kB 52.3 MB/s 
     |████████████████████████████████| 127 kB 54.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
import torch
from torch import cuda
import transformers
from transformers import RobertaModel, AutoModelForMaskedLM, FillMaskPipeline
from transformers import RobertaTokenizerFast
from transformers import TrainerCallback
import datasets
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelBinarizer

from collections import ChainMap
import random
from tqdm import tqdm
import shutil
from datetime import datetime
import json
import os

In [6]:
import multiprocessing

core_count = multiprocessing.cpu_count() # Count the number of cores in a computer
core_count

2

In [7]:
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cpu'

### Load Tokenizer



In [25]:
tokenizer = RobertaTokenizerFast.from_pretrained("Epidot/TwitchLeagueBert-500k")
model = AutoModelForMaskedLM.from_pretrained("Epidot/TwitchLeagueBert-500k")

Downloading:   0%|          | 0.00/462 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/429k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.97M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/636 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

In [28]:
mask_pipe = FillMaskPipeline(model, tokenizer)

### Imaginary examples



In [16]:
mask_pipe(["PogChamp this game was <mask>"])

[{'score': 0.129913792014122,
  'sequence': 'PogChamp this game was amazing',
  'token': 6456,
  'token_str': ' amazing'},
 {'score': 0.129644975066185,
  'sequence': 'PogChamp this game was sick',
  'token': 5472,
  'token_str': ' sick'},
 {'score': 0.06249283626675606,
  'sequence': 'PogChamp this game was awesome',
  'token': 12154,
  'token_str': ' awesome'},
 {'score': 0.05848356708884239,
  'sequence': 'PogChamp this game was insane',
  'token': 6411,
  'token_str': ' insane'},
 {'score': 0.03873783349990845,
  'sequence': 'PogChamp this game was crazy',
  'token': 7554,
  'token_str': ' crazy'}]

In [30]:
mask_pipe(["LUL this game was <mask>"])

[{'score': 0.06814538687467575,
  'sequence': 'LUL this game was fun',
  'token': 1807,
  'token_str': ' fun'},
 {'score': 0.05917441099882126,
  'sequence': 'LUL this game was awful',
  'token': 10536,
  'token_str': ' awful'},
 {'score': 0.05752800405025482,
  'sequence': 'LUL this game was crazy',
  'token': 7554,
  'token_str': ' crazy'},
 {'score': 0.04410829022526741,
  'sequence': 'LUL this game was over',
  'token': 1550,
  'token_str': ' over'},
 {'score': 0.04276535287499428,
  'sequence': 'LUL this game was trash',
  'token': 1954,
  'token_str': ' trash'}]

In [32]:
mask_pipe(["THE BEST IS <mask> AND THE OTHERS ARE <mask>"])

[[{'score': 0.03928021714091301,
   'sequence': '<s>THE BEST IS BRAZIL AND THE OTHERS ARE<mask></s>',
   'token': 3055,
   'token_str': ' BRAZIL'},
  {'score': 0.03532353416085243,
   'sequence': '<s>THE BEST IS OTHERS AND THE OTHERS ARE<mask></s>',
   'token': 20269,
   'token_str': ' OTHERS'},
  {'score': 0.03504947945475578,
   'sequence': '<s>THE BEST IS LPL AND THE OTHERS ARE<mask></s>',
   'token': 1422,
   'token_str': ' LPL'},
  {'score': 0.020053956657648087,
   'sequence': '<s>THE BEST IS FAKER AND THE OTHERS ARE<mask></s>',
   'token': 1099,
   'token_str': ' FAKER'},
  {'score': 0.018589677289128304,
   'sequence': '<s>THE BEST IS BETTER AND THE OTHERS ARE<mask></s>',
   'token': 2358,
   'token_str': ' BETTER'}],
 [{'score': 0.2502993047237396,
   'sequence': '<s>THE BEST IS<mask> AND THE OTHERS ARE TRASH</s>',
   'token': 1000,
   'token_str': ' TRASH'},
  {'score': 0.05845227837562561,
   'sequence': '<s>THE BEST IS<mask> AND THE OTHERS ARE BAD</s>',
   'token': 966,
   

In [33]:
mask_pipe(["NA <mask> EU"], )

[{'score': 0.6753143668174744,
  'sequence': 'NA > EU',
  'token': 563,
  'token_str': ' >'},
 {'score': 0.05331704393029213,
  'sequence': 'NA = EU',
  'token': 732,
  'token_str': ' ='},
 {'score': 0.03790123388171196,
  'sequence': 'NA < EU',
  'token': 379,
  'token_str': ' <'},
 {'score': 0.03292115405201912,
  'sequence': 'NA + EU',
  'token': 1358,
  'token_str': ' +'},
 {'score': 0.020701009780168533,
  'sequence': 'NA VS EU',
  'token': 1089,
  'token_str': ' VS'}]

In [34]:
mask_pipe(["LUL mods are trying to ban us <mask>"])

[{'score': 0.593178391456604,
  'sequence': 'LUL mods are trying to ban us LUL',
  'token': 216,
  'token_str': ' LUL'},
 {'score': 0.1198589876294136,
  'sequence': 'LUL mods are trying to ban us all',
  'token': 997,
  'token_str': ' all'},
 {'score': 0.017600689083337784,
  'sequence': 'LUL mods are trying to ban us monkaS',
  'token': 721,
  'token_str': ' monkaS'},
 {'score': 0.01589123345911503,
  'sequence': 'LUL mods are trying to ban us!',
  'token': 5,
  'token_str': '!'},
 {'score': 0.014292033389210701,
  'sequence': 'LUL mods are trying to ban us?',
  'token': 35,
  'token_str': '?'}]

In [35]:
mask_pipe(["Who chooses <mask> over Fizz? OMG"])

[{'score': 0.15918388962745667,
  'sequence': 'Who chooses Fizz over Fizz? OMG',
  'token': 29733,
  'token_str': ' Fizz'},
 {'score': 0.06211470812559128,
  'sequence': 'Who chooses fizz over Fizz? OMG',
  'token': 16620,
  'token_str': ' fizz'},
 {'score': 0.029678845778107643,
  'sequence': 'Who chooses it over Fizz? OMG',
  'token': 545,
  'token_str': ' it'},
 {'score': 0.02958330512046814,
  'sequence': 'Who chooses Viktor over Fizz? OMG',
  'token': 17197,
  'token_str': ' Viktor'},
 {'score': 0.02709672972559929,
  'sequence': 'Who chooses Syndra over Fizz? OMG',
  'token': 14210,
  'token_str': ' Syndra'}]

In [36]:
mask_pipe(["Wait for it PauseChamp"])

[{'score': 0.19765552878379822,
  'sequence': 'Wait for it?',
  'token': 35,
  'token_str': '?'},
 {'score': 0.06030958890914917,
  'sequence': 'Wait for it!',
  'token': 5,
  'token_str': '!'},
 {'score': 0.047374553978443146,
  'sequence': 'Wait for it LUL',
  'token': 216,
  'token_str': ' LUL'},
 {'score': 0.04305411875247955,
  'sequence': 'Wait for it Pog',
  'token': 260,
  'token_str': ' Pog'},
 {'score': 0.042011674493551254,
  'sequence': 'Wait for it.',
  'token': 18,
  'token_str': '.'}]

### Imaginary examples 500k



In [29]:
mask_pipe(["PogChamp this game was <mask>"])

[{'score': 0.1412344127893448,
  'sequence': 'PogChamp this game was insane',
  'token': 6411,
  'token_str': ' insane'},
 {'score': 0.12279930710792542,
  'sequence': 'PogChamp this game was amazing',
  'token': 6456,
  'token_str': ' amazing'},
 {'score': 0.08581353724002838,
  'sequence': 'PogChamp this game was crazy',
  'token': 7554,
  'token_str': ' crazy'},
 {'score': 0.05686958506703377,
  'sequence': 'PogChamp this game was intense',
  'token': 23691,
  'token_str': ' intense'},
 {'score': 0.04588109999895096,
  'sequence': 'PogChamp this game was hype',
  'token': 6301,
  'token_str': ' hype'}]

In [ ]:
mask_pipe(["LUL this game was <mask>"])

[{'score': 0.06949596852064133,
  'sequence': 'LUL this game was terrible',
  'token': 6982,
  'token_str': ' terrible'},
 {'score': 0.058566413819789886,
  'sequence': 'LUL this game was sad',
  'token': 3984,
  'token_str': ' sad'},
 {'score': 0.04993882030248642,
  'sequence': 'LUL this game was over',
  'token': 1550,
  'token_str': ' over'},
 {'score': 0.048178497701883316,
  'sequence': 'LUL this game was awful',
  'token': 10536,
  'token_str': ' awful'},
 {'score': 0.043460071086883545,
  'sequence': 'LUL this game was pathetic',
  'token': 8829,
  'token_str': ' pathetic'}]

In [ ]:
mask_pipe(["THE BEST IS <mask> AND THE OTHERS"])

[{'score': 0.0510927215218544,
  'sequence': 'THE BEST IS FAKER AND THE OTHERS',
  'token': 1099,
  'token_str': ' FAKER'},
 {'score': 0.05018412321805954,
  'sequence': 'THE BEST IS LCK AND THE OTHERS',
  'token': 1200,
  'token_str': ' LCK'},
 {'score': 0.04398201033473015,
  'sequence': 'THE BEST IS LPL AND THE OTHERS',
  'token': 1422,
  'token_str': ' LPL'},
 {'score': 0.04033534228801727,
  'sequence': 'THE BEST IS CHOVY AND THE OTHERS',
  'token': 3410,
  'token_str': ' CHOVY'},
 {'score': 0.025984354317188263,
  'sequence': 'THE BEST IS UZI AND THE OTHERS',
  'token': 3382,
  'token_str': ' UZI'}]

In [37]:
mask_pipe(["NA <mask> EU"], )

[{'score': 0.6753143668174744,
  'sequence': 'NA > EU',
  'token': 563,
  'token_str': ' >'},
 {'score': 0.05331704393029213,
  'sequence': 'NA = EU',
  'token': 732,
  'token_str': ' ='},
 {'score': 0.03790123388171196,
  'sequence': 'NA < EU',
  'token': 379,
  'token_str': ' <'},
 {'score': 0.03292115405201912,
  'sequence': 'NA + EU',
  'token': 1358,
  'token_str': ' +'},
 {'score': 0.020701009780168533,
  'sequence': 'NA VS EU',
  'token': 1089,
  'token_str': ' VS'}]

In [ ]:
mask_pipe(["LUL mods are trying to ban us <mask>"])

[{'score': 0.765400230884552,
  'sequence': 'LUL mods are trying to ban us LUL',
  'token': 216,
  'token_str': ' LUL'},
 {'score': 0.018221288919448853,
  'sequence': 'LUL mods are trying to ban us mods',
  'token': 2285,
  'token_str': ' mods'},
 {'score': 0.011678830720484257,
  'sequence': 'LUL mods are trying to ban us monkaS',
  'token': 721,
  'token_str': ' monkaS'},
 {'score': 0.01101723313331604,
  'sequence': 'LUL mods are trying to ban us all',
  'token': 997,
  'token_str': ' all'},
 {'score': 0.00998876616358757,
  'sequence': 'LUL mods are trying to ban us?',
  'token': 35,
  'token_str': '?'}]

In [ ]:
mask_pipe(["Who chooses <mask> over Fizz? OMG"])

[{'score': 0.07833434641361237,
  'sequence': 'Who chooses all over Fizz? OMG',
  'token': 997,
  'token_str': ' all'},
 {'score': 0.030908629298210144,
  'sequence': 'Who chooses Fizz over Fizz? OMG',
  'token': 29733,
  'token_str': ' Fizz'},
 {'score': 0.023928476497530937,
  'sequence': 'Who chooses it over Fizz? OMG',
  'token': 545,
  'token_str': ' it'},
 {'score': 0.022759059444069862,
  'sequence': 'Who chooses LB over Fizz? OMG',
  'token': 6014,
  'token_str': ' LB'},
 {'score': 0.019884660840034485,
  'sequence': 'Who chooses Azir over Fizz? OMG',
  'token': 7965,
  'token_str': ' Azir'}]